In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import re
import calendar
from datetime import datetime, timedelta

# API Access Key and Token

In [2]:
my_key = "8f8e4f67-63b7-47c6-8ce8-b9fb0e4a09b1"
headers = {'AccessKey': my_key, 'User-Agent': 'Mozilla/5.0'}

def get_token():
    response = requests.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action",
                             headers=headers, data=headers, json=headers)

    if response.status_code == 200:
        return response.json()["Result"]
    else:
        return "Error"

my_token = get_token()

# Median Rental

In [3]:
######################## extract ########################
headers = {'AccessKey': my_key, "Token": my_token, 'User-Agent': 'Mozilla/5.0'}
response = requests.get("https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Rental_Median", headers=headers)
if response.status_code == 200:
    result = response.json()
else:
    print("Unsuccessful")

median_rentals = pd.DataFrame()
for ele in result["Result"]:
    # print(ele.keys())
    curr_details = pd.DataFrame(ele["rentalMedian"]) # refPeriod, psg75, median, psf25, district
    curr_details["street"] = ele["street"]
    curr_details["x"] = ele["x"]
    curr_details["y"] = ele["y"]
    curr_details["project"] = ele["project"]
    median_rentals = pd.concat([median_rentals, curr_details], axis=0)

######################## transform ########################
# casting the correct datatypes
for col in ['psf75', 'median', 'psf25', 'x', 'y']:
    median_rentals[col] = median_rentals[col].astype(float)

median_rentals = median_rentals.reset_index(drop=True)
median_rentals 

,refPeriod,psf75,median,psf25,district,street,x,y,project
0,2021Q4,2.40,2.17,1.98,23,HILLVIEW AVENUE,19806.57039,37591.36252,MERAWOODS
1,2021Q2,2.28,2.06,1.86,23,HILLVIEW AVENUE,19806.57039,37591.36252,MERAWOODS
2,2022Q1,2.43,2.07,1.94,23,HILLVIEW AVENUE,19806.57039,37591.36252,MERAWOODS
3,2022Q3,2.94,2.60,2.38,23,HILLVIEW AVENUE,19806.57039,37591.36252,MERAWOODS
4,2022Q4,3.25,3.01,2.80,23,HILLVIEW AVENUE,19806.57039,37591.36252,MERAWOODS
...,...,...,...,...,...,...,...,...,...
7304,2022Q3,6.00,4.97,4.79,20,SIN MING ROAD,28255.63458,37271.68720,THOMSON V TWO
7305,2022Q2,6.83,6.54,5.89,08,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER
7306,2022Q1,6.63,6.39,6.01,08,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER
7307,2023Q4,7.43,7.15,6.49,08,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER


In [4]:
median_rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7309 entries, 0 to 7308
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   refPeriod  7309 non-null   object 
 1   psf75      7309 non-null   float64
 2   median     7309 non-null   float64
 3   psf25      7309 non-null   float64
 4   district   7309 non-null   object 
 5   street     7309 non-null   object 
 6   x          7309 non-null   float64
 7   y          7309 non-null   float64
 8   project    7309 non-null   object 
dtypes: float64(5), object(4)
memory usage: 514.0+ KB


# Rental Contracts

## Extract

In [50]:
######################## extract ########################
# get list of refPeriod to extract
# choosing refPeriod that is same as rental median data, which depends on when the data is refreshed
today = datetime.today().date()
latest_q = pd.Timestamp(today).quarter - 1
c = calendar.Calendar(firstweekday=calendar.MONDAY)
data_refresh_month = {1: 4, 2:7, 3:10, 4:1} # key=quarter, value=month

year = today.year; month = data_refresh_month[latest_q]

monthcal = c.monthdatescalendar(year,month)
fourth_friday = [day for week in monthcal for day in week if \
                day.weekday() == calendar.FRIDAY and \
                day.month == month][3]
if today > fourth_friday:
    lst_periods = list(str(x) for x in pd.period_range(end=today - timedelta(days=91), periods=12, freq="Q"))
else:
    lst_periods = list(str(x) for x in pd.period_range(end=today - timedelta(days=91*2), periods=12, freq="Q"))

# extract data from API
def get_rental_contracts_per_refPeriod(period):
    """period should be a string e.g. '2014Q1' means 2014 1st quarter"""
    refPeriod = period[2:].lower() # format should be like 14q1 instead of 2014Q1
    # refPeriod is mandatory parameter
    headers = {'AccessKey': my_key, "Token": my_token, 'User-Agent': 'Mozilla/5.0'}
    response = requests.get(f"https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Rental&refPeriod={refPeriod}", headers=headers)
    if response.status_code == 200:
        result = response.json()
    else:
        print("Unsuccessful")
    rental_contracts_per_refPeriod = pd.DataFrame()
    for ele in result["Result"]:
        curr_details = pd.DataFrame(ele["rental"]) # areaSqm	leaseDate	propertyType	district	areaSqft	noOfBedRoom	rent
        curr_details["street"] = ele["street"]

        # some dont have x and y fields. Set to 0.
        try:
            curr_details["x"] = float(ele["x"])
        except KeyError:
            curr_details["x"] = 0
        try:
            curr_details["y"] = float(ele["y"])
        except KeyError:
            curr_details["y"] = 0
        curr_details["project"] = ele["project"]
        rental_contracts_per_refPeriod = pd.concat([rental_contracts_per_refPeriod, curr_details], axis=0)
    rental_contracts_per_refPeriod["refPeriod"] = period
    return rental_contracts_per_refPeriod.reset_index(drop=True)

rental_contracts = pd.DataFrame()
for period in lst_periods:
    print(period)
    rental_contracts = pd.concat([rental_contracts, get_rental_contracts_per_refPeriod(period)], axis=0)
rental_contracts

2021Q1
2021Q2
2021Q3
2021Q4
2022Q1
2022Q2
2022Q3
2022Q4
2023Q1
2023Q2
2023Q3
2023Q4


,areaSqm,leaseDate,propertyType,district,areaSqft,noOfBedRoom,rent,street,x,y,project,refPeriod
0,240-250,0121,Non-landed Properties,15,2600-2700,4,5500,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1
1,120-130,0121,Non-landed Properties,15,1300-1400,3,3700,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1
2,130-140,0221,Non-landed Properties,15,1400-1500,3,4000,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1
3,190-200,0221,Non-landed Properties,15,2100-2200,3,5500,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1
4,130-140,0221,Non-landed Properties,15,1400-1500,3,4000,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1
...,...,...,...,...,...,...,...,...,...,...,...,...
19824,40-50,1123,Non-landed Properties,08,500-600,2,4000,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER,2023Q4
19825,60-70,1123,Non-landed Properties,08,600-700,3,5000,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER,2023Q4
19826,40-50,1123,Non-landed Properties,08,500-600,2,4000,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER,2023Q4
19827,60-70,1223,Non-landed Properties,08,600-700,3,5600,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER,2023Q4


## Transform

In [52]:
######################## transform ########################
# fill NA noOfBedRoom with -1
rental_contracts["noOfBedRoom"] = rental_contracts.apply(lambda row: -1 if row["noOfBedRoom"] == "NA" else row["noOfBedRoom"], axis=1)

# casting the correct datatypes
for col in ['rent', 'x', 'y']:
    rental_contracts[col] = rental_contracts[col].astype(float)

for col in ['noOfBedRoom']:
    rental_contracts[col] = rental_contracts[col].astype(int)

rental_contracts = rental_contracts.reset_index(drop=True)

# areaSqft are numerical intervals that are consecutive and mostly equal
# convert these to a single number (the midpoint of the interval) and use this as a quantitative variable
# there are some unbounded intervals e.g. <=1000, >3000, >8000 which will be replaced with the boundary itself
def format_area(s):
    try:
        lower, upper = s.split("-")
        lower = int(lower)
        upper = int(upper)
        return (lower + upper) / 2
    except ValueError:
        pattern = r"(^<|>=|<=|>)(\d+)"
        match = re.match(pattern, s)
        return float(match.group(2))
rental_contracts["areaSqft_formatted"] = rental_contracts["areaSqft"].apply(format_area)

# change leaseDate to datetime and extract year, quarter, month as new features
rental_contracts["leaseDate"] = rental_contracts["leaseDate"].astype(int)
rental_contracts["leaseDate"] = rental_contracts.apply(lambda row: datetime(int(row["refPeriod"][:4]), row["leaseDate"] // 100, 1), axis=1)
rental_contracts["leaseYear"] = rental_contracts["leaseDate"].dt.year
rental_contracts["leaseQuarter"] = rental_contracts["leaseDate"].dt.quarter
rental_contracts["leaseMonth"] = rental_contracts["leaseDate"].dt.month

# Since the IQR is in per square feet, we will not use areaSqm. 
# Original areaSqft will also not be used
rental_contracts = rental_contracts.drop(columns=["areaSqft", "areaSqm"])
rental_contracts

,leaseDate,propertyType,district,noOfBedRoom,rent,street,x,y,project,refPeriod,areaSqft_formatted,leaseYear,leaseQuarter,leaseMonth
0,2021-01-01,Non-landed Properties,15,4,5500.0,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1,2650.0,2021,1,1
1,2021-01-01,Non-landed Properties,15,3,3700.0,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1,1350.0,2021,1,1
2,2021-02-01,Non-landed Properties,15,3,4000.0,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1,1450.0,2021,1,2
3,2021-02-01,Non-landed Properties,15,3,5500.0,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1,2150.0,2021,1,2
4,2021-02-01,Non-landed Properties,15,3,4000.0,ELLIOT ROAD,38865.17182,32621.92134,ELLIOT AT THE EAST COAST,2021Q1,1450.0,2021,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281601,2023-11-01,Non-landed Properties,08,2,4000.0,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER,2023Q4,550.0,2023,4,11
281602,2023-11-01,Non-landed Properties,08,3,5000.0,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER,2023Q4,650.0,2023,4,11
281603,2023-11-01,Non-landed Properties,08,2,4000.0,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER,2023Q4,550.0,2023,4,11
281604,2023-12-01,Non-landed Properties,08,3,5600.0,PERUMAL ROAD,30477.87296,32796.58779,UPTOWN @ FARRER,2023Q4,650.0,2023,4,12


In [53]:
rental_contracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281606 entries, 0 to 281605
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   leaseDate           281606 non-null  datetime64[ns]
 1   propertyType        281606 non-null  object        
 2   district            281606 non-null  object        
 3   noOfBedRoom         281606 non-null  int32         
 4   rent                281606 non-null  float64       
 5   street              281606 non-null  object        
 6   x                   281606 non-null  float64       
 7   y                   281606 non-null  float64       
 8   project             281606 non-null  object        
 9   refPeriod           281606 non-null  object        
 10  areaSqft_formatted  281606 non-null  float64       
 11  leaseYear           281606 non-null  int32         
 12  leaseQuarter        281606 non-null  int32         
 13  leaseMonth          281606 no